In [12]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import notebook_helper
import imodelsx.process_results
import sys
from mprompt.modules.fmri_module import get_test_ngrams
sys.path.append('../experiments/')
# results_dir = '../results/feb12_fmri_sweep/'
results_dir = '/home/chansingh/mntv1/mprompt/feb12_fmri_sweep_gen_template1/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
r = imodelsx.process_results.get_results_df(results_dir, use_cached=True)
r['top_ngrams_test'] = r.apply(lambda row: get_test_ngrams(voxel_num_best=row.module_num)[:20], axis=1)
r.to_pickle('../results/results_fmri.pkl')

In [54]:
r['explanation_init_ngrams'] = r['explanation_init_ngrams'].apply(lambda x: x[:20])
tab = (
    r
    .sort_values(by='top_score_synthetic', ascending=False)
    .filter([
        'module_num', 'top_score_synthetic', 'fmri_test_corr',
        'top_explanation_init_strs', 'explanation_init_ngrams',
        'top_ngrams_test',
        'explanation_init_strs',
    ])
    .round(3)
    .rename(
        columns={
            'top_explanation_init_strs': 'Explanation',
            'explanation_init_ngrams': 'Top ngrams (module)',
            'module_num': 'Voxel',
            'top_ngrams_test': 'Top ngrams (test)',
            'top_score_synthetic': 'Synthetic score',
            'fmri_test_corr': 'Correlation (test)',
            'explanation_init_strs': 'Explanation candidates',
        }
    )
)
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    display(
        tab.head(20)
    )
    tab.to_csv('../results/results_fmri.csv', index=False)

,Voxel,Synthetic score,Correlation (test),Explanation,Top ngrams (module),Top ngrams (test),Explanation candidates
188,143,0.051,0.628,movement or action,"[started playing, up in new, grew up in, was replications of, was going to, is going to, up in two, started to scream, story begins in, up in dartmouth, up in providence, are going to, went to high, trained to eject, was not going, grew up on, grew up near, item originated from, thought yeah yeah, really dismayed to]","[was ok because, had my cigarettes, i know it's, the way to, have my cigarettes, a little boy, say wow that's, stores and emergecenters, where were you, and she very, of the television, so what i, he is beautiful, that what uh, lit up so, when i needed, it a bad, what our lives, the street street, lives we're both]","[movement or action, action or movement, such as starting, going, playing, screaming, etc, an action or event that is happening or has happened, movement or change, either physically or mentally]"
178,119,0.050,0.635,interaction between two or more people,"[turned to me, turns to me, president looks into, she had this, some pubis sticking, says tricia, problem she, been raised that, and says tricia, he s he, i had this, she bought that, i turned to, sources the the, president put in, shy was cause, pokes her head, the church, that caught my, shy was]","[i'm going home, head falling off, and she also, i know it's, a little boy, girl lives there, the neighborhood this, cool job washing, i hope so, i was crying, ch ch ch, lovely and there, we're loving the, be a vet, dear you don't, her and just, door there's some, oh that's almost, mm sweet kiss, you've had them]","[interaction between two or more people, relationships, turning or looking towards someone or something, people interacting with each other, people and relationships]"
42,124,0.048,0.633,"""um.""","[in the leg, um mm mm, and and um, do drum, um and um, to draw, how to draw, in between um, um and and, cliche um image, around and um, in other words, and um and, lothario, bla and um, be and um, who does the, playing an instrument, it was odd, artists model]","[i ran i, when i see, gone and then, she tells me, cool job washing, was gonna happen, it ok and, quit that boyfriend, and emergecenters and, you just run, ok great and, door's right there, to jump out, person comes closer, does the opposite, on she has, parlor so um, first she digs, on my breath, know like medium]","[""um."", ""um""]"
199,114,0.045,0.636,"people and activities, such as orthopedist, confidant, conscientious objector, women, make up, and calligrapher","[another orthopedist, orthopedist, another orthopedist whom, orthopedist whom, confidant, to another orthopedist, unannounced, styrofoam plate, unsent greeting cards, who 's paroled, photosensitive epilepsy, styrofoam, a confidant, up unannounced, round styrofoam plate, photosensitive epilepsy so, round styrofoam, treatment of depression, of epilepsy, living with epilepsy]","[i'm like man, disgusting and poison, looking very bad, was fun and, the ground and, or you run, you're halfway there, you got a, she said oh, it ok and, be real nice, was ok because, over you'll be, very very bad, possible future and, know like medium, a little boy, i know it's, stories about our, we got to]","[people and activities, such as orthopedist, confidant, conscientious objector, women, make up, and calligrapher, medical and mental health related topics, people, either professions or activities, medical conditions, treatments, and/or medical professionals, people or things related to medical conditions, such as orthopedists, epilepsy, depression, and conscientious objectors]"
163,108,0.041,0.638,"""got ta"" or ""theme song""","[got to say, something was up, bye bye bye, got ta go, got ta get, got ta do, the theme song, got ta be, got ta you, main rotor blades, so slow, one problem was, downhill main, main rotor blade, one thing that, this was all, wo n't say, bye bye, dingy little rooms, them

In [ ]:
sns.lmplot(x='top_score_synthetic', y='fmri_test_corr', data=r)
plt.show()